In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import statistics as stt
import os
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

class LaneLInesFinder(object):

    def __init__(self):
        # for smooth lines, save points of lines in past frames, then get mean of those points and draw lines
        self.rline_x1s = list()
        self.rline_y1s = list()
        self.rline_x2s = list()
        self.lline_x1s = list()
        self.lline_y1s = list()
        self.lline_x2s = list()
        
    def grayscale(self, img):
        """Applies the Grayscale transform
        This will return an image with only one color channel
        but NOTE: to see the returned image as grayscale
        (assuming your grayscaled image is called 'gray')
        you should call plt.imshow(gray, cmap='gray')"""
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Or use BGR2GRAY if you read an image with cv2.imread()
        # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    def hsvscale(self, img):
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
                            
    def canny(self, img, low_threshold, high_threshold):
        """Applies the Canny transform"""
        return cv2.Canny(img, low_threshold, high_threshold)

    def gaussian_blur(self, img, kernel_size):
        """Applies a Gaussian Noise kernel"""
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

    def region_of_interest(self, img, vertices):
        """
        Applies an image mask.
        
        Only keeps the region of the image defined by the polygon
        formed from `vertices`. The rest of the image is set to black.
        """
        #defining a blank mask to start with
        mask = np.zeros_like(img)   
        
        #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
        if len(img.shape) > 2:
            channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
            
        #filling pixels inside the polygon defined by "vertices" with the fill color    
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        
        #returning the image only where mask pixels are nonzero
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image



    def draw_lines(self, img, lines, color=[255, 0, 0], thickness=15):
        """
        NOTE: this is the function you might want to use as a starting point once you want to 
        average/extrapolate the line segments you detect to map out the full
        extent of the lane (going from the result shown in raw-lines-example.mp4
        to that shown in P1_example.mp4).  
        
        Think about things like separating line segments by their 
        slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
        line vs. the right line.  Then, you can average the position of each of 
        the lines and extrapolate to the top and bottom of the lane.
        
        This function draws `lines` with `color` and `thickness`.    
        Lines are drawn on the image inplace (mutates the image).
        If you want to make the lines semi-transparent, think about combining
        this function with the weighted_img() function below
        """

        lline_x = list()
        lline_y = list()
        rline_x = list()
        rline_y = list()
        lslope = list()
        rslope = list()
        start_roi_y = 450
        h, w, c = img.shape
        for line in lines:
            for x1,y1,x2,y2 in line:
                if (x2-x1) == 0:
                    continue
                slope = ((y2-y1)/(x2-x1))
                if slope > 0.5:
                    rslope.append(slope)
                    rline_x.extend([x1,x2])
                    rline_y.extend([y1,y2])
                elif slope < -0.5:
                    lslope.append(slope)
                    lline_x.extend([x1,x2])
                    lline_y.extend([y1,y2])
                else:
                    continue

        # get mean slope value
        mean_lslope = stt.mean(lslope)
        mean_rslope = stt.mean(rslope)
        
        # calculate x coordinate with mean slope value from 'slope = (y2-y1)/(x2-x1)'
        
        end_right_x = (h - min(rline_y))/mean_rslope + min(rline_x)  # x2
        start_right_x = end_right_x - (h - start_roi_y)/mean_rslope  # x1
        
        end_left_x = (h - min(lline_y))/mean_lslope + max(lline_x)   # x2
        start_left_x = end_left_x - (h - start_roi_y)/mean_lslope    # x1

        if end_right_x > w/2:  # for remove abnormal lines
            self.rline_x1s.extend([start_right_x])
            self.rline_x2s.extend([end_right_x])
            self.rline_y1s.extend([start_roi_y])
            
            if len(self.rline_x1s) > 10:
                rx1 = int(stt.mean(self.rline_x1s[-10:]))
                rx2 = int(stt.mean(self.rline_x2s[-10:]))
                ry1 = int(stt.mean(self.rline_y1s[-10:]))
                ry2 = h
            else:
                rx1 = int(stt.mean(self.rline_x1s))
                rx2 = int(stt.mean(self.rline_x2s))
                ry1 = int(stt.mean(self.rline_y1s))
                ry2 = h

            cv2.line(img, (rx1, ry1), (rx2, ry2), color, thickness)
            
            #print('Right lane: (', min(rline_x), ', ', min(rline_y), ') ', '(', int(end_right_x), ', ', h, ')')
        if end_left_x < w/2:
            self.lline_x1s.extend([start_left_x])
            self.lline_x2s.extend([end_left_x])
            self.lline_y1s.extend([start_roi_y])

            if len(self.lline_x1s) > 10:
                lx1 = int(stt.mean(self.lline_x1s[-10:]))
                lx2 = int(stt.mean(self.lline_x2s[-10:]))
                ly1 = int(stt.mean(self.lline_y1s[-10:]))
                ly2 = h
            else:
                lx1 = int(stt.mean(self.lline_x1s))
                lx2 = int(stt.mean(self.lline_x2s))
                ly1 = int(stt.mean(self.lline_y1s))
                ly2 = h
            
            cv2.line(img, (lx1, ly1), (lx2, ly2), color, thickness)

            #print('Left lane: (', max(lline_x), ', ', min(lline_y), ') ', '(', int(end_left_x), ', ', h, ')')

    def hough_lines(self, img, rho, theta, threshold, min_line_len, max_line_gap):
        """
        `img` should be the output of a Canny transform.
            
        Returns an image with hough lines drawn.
        """
        lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        self.draw_lines(line_img, lines)
        return line_img

    # Python 3 has support for cool math symbols.

    def weighted_img(self, img, initial_img, α=0.8, β=1., λ=0.):
        """
        `img` is the output of the hough_lines(), An image with lines drawn on it.
        Should be a blank image (all black) with lines drawn on it.
        
        `initial_img` should be the image before any processing.
        
        The result image is computed as follows:
        
        initial_img * α + img * β + λ
        NOTE: initial_img and img must be the same shape!
        """
        return cv2.addWeighted(initial_img, α, img, β, λ)

    def normalize_image(self, image):
        imshape = image.shape
        norm = np.zeros(imshape, np.float32)
        norm_rgb = np.zeros(imshape, np.uint8)
        
        b = image[:,:,0]
        g = image[:,:,1]
        r = image[:,:,2]
        
        sum = b + g + r
        
        norm[:,:,0] = b/sum*255.0
        norm[:,:,1] = g/sum*255.0
        norm[:,:,2] = r/sum*255.0
        
        norm_rgb = cv2.convertScaleAbs(norm)
        return norm_rgb

    def process_image(self, image):
        #gray = self.hsvscale(image)
        img = np.copy(image)
        gray = self.normalize_image(img)
        
        # Define a kernel size and apply Gaussian smoothing
        kernel_size = 7
        blur_gray = self.gaussian_blur(gray,kernel_size)

        # Define our parameters for Canny and apply
        low_threshold = 35
        high_threshold = 180
        edges = self.canny(blur_gray, low_threshold, high_threshold)

        imshape = image.shape
        vertices = np.array([[(300,imshape[0]-100),(600, 450), (750, 450), (1200, imshape[0]-100)]], dtype=np.int32)
        masked_edges = self.region_of_interest(edges, vertices)

        # Define the Hough transform parameters
        # Make a blank the same size as our image to draw on
        rho = 1 # distance resolution in pixels of the Hough grid
        theta = np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 35    # minimum number of votes (intersections in Hough grid cell)
        min_line_len = 5  #minimum number of pixels making up a line
        max_line_gap = 2  # maximum gap in pixels between connectable line segments

        # Run Hough on edge detected image
        line_image = self.hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)

        # Create a "color" binary image to combine with line image
        color_edges = np.dstack((masked_edges, masked_edges, masked_edges)) 

        # Draw the lines on the edge image
        lines_edges = self.weighted_img(line_image, image, 0.7, 1, 0)

        return lines_edges


# Main procedure
process_func = LaneLInesFinder()

video_dir = 'test_videos'
video_outdir = 'test_videos_output'

video_list = [ 'challenge.mp4' ]
video_out_list = [ 'challenge_output.mp4' ]


new_clip_output = video_outdir + '/' + video_out_list[0]
test_clip = VideoFileClip(video_dir + '/' + video_list[0])

new_clip = test_clip.fl_image(process_func.process_image)
%time new_clip.write_videofile(new_clip_output, audio=False)

print('Output: ', video_outdir + '/' + video_out_list[0])
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))

C:\Anaconda3\lib\site-packages\skimage\filter\__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '



MoviePy: building video file test_videos_output/challenge_output.mp4
----------------------------------------

Writing video into test_videos_output/challenge_output.mp4
|----------| 0/241   0% [elapsed: 00:00 left: ?, ? iters/sec]

C:\Anaconda3\lib\site-packages\moviepy\audio\io\readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:204: RuntimeWarning: divide by zero encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:204: RuntimeWarning: invalid value encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:205: RuntimeWarning: divide by zero encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:205: RuntimeWarning: invalid value encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:206: RuntimeWarning: divide by zero encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:206: RuntimeWarning: invalid value encountered in true_divide
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:204: RuntimeWarning: divide by zero encountered in true_d

Done writing video in test_videos_output/challenge_output.mp4 !        
Your video is ready !
Wall time: 18.4 s
Output:  test_videos_output/challenge_output.mp4
